In [ ]:
%load_ext autoreload
from pathlib import Path

import torch

from src.data_loader_cifar import get_cifar_dataloaders
import src.model_constructor as constructor
import itertools

In [2]:
%autoreload 2
def models_iterator(depths, filters_sizes, optimizers, drops, lrs):
    models_to_train = []
    for depth in depths:
        fs = filters_sizes[str(depth)]
        d = drops[str(depth)]
        configurations = list(itertools.product(fs, optimizers, d, lrs))

        for config in configurations:
            filters_size, optimizer, drop, lr = config

            nr_filters = filters_size[:depth]
            conv_layers = constructor.Conv(nr_conv=depth, nr_filters=nr_filters, maxpool_batchnorm=True)
            fc_size = filters_size[depth:]
            act_fun = ["ReLU"] * depth
            dropouts = drop
            fc_layers = constructor.FC(
                nr_fc=depth,
                fc_size=fc_size,
                act_funs=act_fun,
                dropouts=dropouts,
                in_features=conv_layers.finaldim,
                num_classes=10,
                batchnorm=True,
            )

            # Create the model using the CNN constructor
            model = constructor.CNN(
                conv_layers=conv_layers, fc_layers=fc_layers, num_classes=10, lr=lr, optim=optimizer
            )

            # Store the model and its parameters in the list
            model_info = {"name": f"{model.name}", "model": model, "params": {"lr": lr, "optimizer": optimizer}}

            models_to_train.append(model_info)

    return models_to_train


depths = [2, 4]
filters_sizes = {
    "2": [[8, 16, 160, 80], [32, 64, 320, 160]],
    "4": [[4, 8, 16, 32, 200, 200, 160, 80], [8, 16, 32, 64, 400, 320, 160, 80]],
}
lrs = [0.01, 0.001, "scheduler"]
drops = {"2": [[0.0, 0.0], [0.5, 0.2]], "4": [[0.0] * 4, [0.5, 0.3, 0.3, 0.2]]}
optimizers = ["adam", "sgd"]


models_to_train = models_iterator(depths, filters_sizes, optimizers, drops, lrs)

In [3]:
%autoreload 2
import warnings
warnings.filterwarnings("ignore")

name = models_to_train[0]['name']
model = models_to_train[0]['model']

model_path = f"models/{name}"
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"Device: {device}")

trained_model = models_to_train[0]['model']
untrained_model = models_to_train[0]['model']

trained_model.load_state_dict(torch.load(model_path+'/trained.pt', map_location=device))
untrained_model.load_state_dict(torch.load(model_path+'/untrained.pt', map_location=device))

trained_model.to(device)
untrained_model.to(device)

train_dataloader, val_dataloader, test_dataloader = get_cifar_dataloaders(path=Path("data/processed/cifar"), batch_size=64, device=device)

Device: cuda


In [4]:
trained_model.eval()
correct = total = 0

with torch.no_grad():
    for X, y in train_dataloader:
        preds = trained_model(X).argmax(dim=1)
        correct += (preds == y).sum().item()
        total += y.size(0)

accuracy = correct / total

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

In [ ]:
from src.measures_sharpeness import calculate_pac_bayes_metrics

metrics = calculate_pac_bayes_metrics(trained_model, untrained_model, train_dataloader,)